In [7]:
import pandas as pd
import networkx as nx

In [13]:
ratings = pd.read_csv("/tmp/fastapi/ratings.csv")

In [14]:
edges = (
    ratings
    .assign(
        user_id=lambda df: df["user_id"].map(lambda x: "U" + str(x)),
        item_id=lambda df: df["item_id"].map(lambda x: "I" + str(x))
    )
    .filter(["user_id", "item_id"])
)

In [21]:
edges.itertuples(index=False)

In [17]:
graph = nx.Graph()

In [24]:
graph

In [27]:
import duckdb

In [37]:
_connection = duckdb.connect(":memory:")

queries = [
    """
    PRAGMA threads=1;
    """,
    """
    CREATE TABLE users (
        user_id INT PRIMARY KEY,
        gender VARCHAR(2),
        age SMALLINT,
        occupation SMALLINT,
        zip_code VARCHAR(10)
    );
    """,
    f"""
    COPY users FROM '/tmp/fastapi/users.csv' (
        AUTO_DETECT FALSE, HEADER TRUE
    );
    """,
    """
    CREATE TABLE movies (
        item_id INT PRIMARY KEY,
        title VARCHAR(100),
        genres VARCHAR(100)
    );
    """,
    f"""
    COPY movies FROM '/tmp/fastapi/movies.csv' (
        AUTO_DETECT FALSE, HEADER TRUE
    );
    """
]

for query in queries:
    _connection.execute(query)

In [30]:
item_id = 1

In [45]:
_connection.execute(
    f"""
    SELECT item_id, title
    FROM movies
    WHERE item_id = {item_id}
    """
).fetchone()

(1, 'Toy Story (1995)')

In [38]:
item_ids = [1, 2, 3, 4]

In [41]:
predicates = " OR ".join([f"item_id={item_id}" for item_id in item_ids])

In [43]:
predicates

'item_id=1 OR item_id=2 OR item_id=3 OR item_id=4'

In [46]:
_connection.execute(
    f"""
    SELECT item_id, title
    FROM movies
    WHERE {predicates}
    """
).fetchall()

[(1, 'Toy Story (1995)'),
 (2, 'Jumanji (1995)'),
 (3, 'Grumpier Old Men (1995)'),
 (4, 'Waiting to Exhale (1995)')]